In [2]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

In [12]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type="GPU")

In [15]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [16]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [46]:
data_dir = 'I:\\datasets\\face'
for directory in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, directory)):
        for file in os.listdir(os.path.join(data_dir,directory)):
            EX_PATH = os.path.join(data_dir, directory, file)
            NEW_PATH = os.path.join(NEG_PATH, file)
            os.replace(EX_PATH, NEW_PATH)

In [47]:
import uuid
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

'data\\anchor\\381e5b28-749f-11ec-8f49-74d83e508ff3.jpg'

In [48]:
cap = cv2.VideoCapture(2)

while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[200:, 400:900, :]
    cv2.inshow("image collection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("a"):
        imgname = os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
    
    if cv2.waitKey(1) & 0xFF == ord("p"):
        imgname = os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
        
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [55]:
def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1, 2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100), np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100,seed=(np.random.randint(100), np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9, upper=1, seed=(np.random.randint(100), np.random.randint(100)))
        data.append(img)
    return data

In [ ]:
img_path = os.path.join(ANC_PATH, '  ')
img = cv2.imread(img_path)
augumented_images = data_aug(img)

plt.imshow(augumented_images[5])

In [ ]:
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = cv2.imread(img_path)
    augumented_images = data_aug(img)
    
    for image = augumented_images:
        cv2.imwrite(os.path.join(POS_PATH, "augumented-()".format(uuid.uuid1())), image.numpy())

In [ ]:
anchor = tf.data.Dataset.list_files(ANC_PATH + '/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH + '/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH + '/*.jpg').take(300)


In [ ]:
dir_test = anchor.as_numpy_iterator()

In [58]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    
    img = tf.image.resize(img, (100, 100))
    img = img / 255.
    return img

In [ ]:
img = preprocess('data/anchor/  .jpg')

In [59]:
img.numpy().min()

AttributeError: 'NoneType' object has no attribute 'numpy'

In [60]:
img.numpy().max()

AttributeError: 'NoneType' object has no attribute 'numpy'

In [61]:
# 创建带有标签的数据集

positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))

negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))


data = positives.concatenate(negatives)

                                

NameError: name 'anchor' is not defined

In [62]:
samples = data.as_numpy_iterator()

NameError: name 'data' is not defined

In [ ]:
samples.next()